In [25]:
import ee
import multiprocessing
from retry import retry
import requests
import shutil

In [2]:
ee.Initialize(project='ee-jonstar', opt_url='https://earthengine-highvolume.googleapis.com')

In [26]:
def getRequests():
    """Generates a list of work items to be downloaded.
    Produces 1000 random points in each of the RESOLVE ecoregions in the ROI.
    """
    # Get our ROI from the GAUL regions
    gaul = ee.FeatureCollection('FAO/GAUL/2015/level1')
    roi = gaul.filter('ADM1_NAME == "Colorado"').first().geometry()
    
    # To stratify by RESOLVE ecoregion, paint the ECO_IDs into an image.
    resolve = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017')
    ecoregions = (resolve.reduceToImage(['ECO_ID'], ee.Reducer.first())
                .clip(roi).rename('ECO_ID'))
    points = ecoregions.stratifiedSample(
      numPoints=1000,
      classBand='ECO_ID',
      region=roi,
      scale=100,
      geometries=True)
    
    return points.aggregate_array('.geo').getInfo()

In [27]:
@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
    """Handle the HTTP requests to download an image."""
    
    # Generate the desired image from the given point.
    point = ee.Geometry.Point(point['coordinates'])
    region = point.buffer(127).bounds()
    image = (ee.ImageCollection('USDA/NAIP/DOQQ')
           .filterBounds(region)
           .filterDate('2019', '2020')
           .mosaic()
           .clip(region)
           .select('R', 'G', 'B'))
    print('made an image')
    
    # Fetch the URL from which to download the image.
    url = image.getThumbURL({
      'region': region,
      'dimensions': '256x256',
      'format': 'png'})
    print('got URL')
    
    # Handle downloading the actual pixels.
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        raise r.raise_for_status()
    print('request successful')
    
    filename = 'tile_%05d.png' % index
    with open(filename, 'wb') as out_file:
        print('file opened')
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", index)

In [23]:
items[0]

{'geodesic': False,
 'type': 'Point',
 'coordinates': [-106.75893246825835, 40.759708543997114]}

In [28]:
getResult(1, items[1])

made an image
got URL
request successful
file opened
Done:  1


In [12]:
items = getRequests()

pool = multiprocessing.Pool(25)
pool.starmap(getResult, enumerate(items))
pool.close()
pool.join()

Process SpawnPoolWorker-334:
Process SpawnPoolWorker-331:
Process SpawnPoolWorker-329:
Process SpawnPoolWorker-337:
Process SpawnPoolWorker-330:
Process SpawnPoolWorker-328:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process SpawnPoolWorker-332:
Traceback (most recent call last):
Traceback (most recent call last):
Process SpawnPoolWorker-327:
  File "/opt/anaconda3/envs/heat/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/heat/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/heat/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/anaconda3/envs/heat/lib/python3.12/multiprocessing/queues.py", line 389, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
Attri

KeyboardInterrupt: 

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/anaconda3/envs/heat/lib/python3.12/multiprocessing/__init__.py", line 16, in <module>
Process SpawnPoolWorker-452:
    from . import context
  File "/opt/anaconda3/envs/heat/lib/python3.12/multiprocessing/context.py", line 6, in <module>
Traceback (most recent call last):
    from . import reduction
  File "/opt/anaconda3/envs/heat/lib/python3.12/multiprocessing/reduction.py", line 33, in <module>
    class ForkingPickler(pickle.Pickler):
KeyboardInterrupt
  File "/opt/anaconda3/envs/heat/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/heat/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/heat/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/anaconda3/envs/heat/lib/python3.12/multiprocessing/queu